# Boosting

## AdaBoost
- 이진분류에서만 사용가능 -> y의 값 : -1, 1
- 그 전단계 위원회의 성능을 보완하여 위원회에 개별 모형을 추가하는 식으로 위원회 성능 보완
- 최종 예측시에는 개별 약 분류기들에 각각 가중치를 적용, 조합하여 얻음  
- 여러 개별 후보들 : 모든 feature 
- 그 중에서 위원회에 넣을 개별 모형을 선별하는 방법
    1. 틀리게 예측한 데이터의 가중치를 합한 값을 손실함수 L로 사용 -> 이것을 최소화하는 모형이 다음 모형으로 선택
  
- 틀린 데이터에 대해 적용할 가중치 = > ln((1-e)/e)
- 처음에 모든 data에 대해 같은 weight 값 -> 1/데이터 개수의 값을 가짐.
https://velog.io/@sangyeop/AdaBoost-%ED%95%9C-%EB%88%88%EC%97%90-%EC%95%8C%EC%95%84%EB%B3%B4%EA%B8%B0

## Gradient Boost
- 분류 및 회귀 모든 문제에 사용 가능
- 현재까지 학습된 모델의 약점을 드러내는 역할을 하고, 다음 모델이 그걸 중점적으로 보완해서 성능을 Boosting 하는 방법
- 분류 및 회귀 문제에 상관없이 회귀분석 모형을 사용.
- 맨 처음 모든 데이터의 예측값으로 실제 값의 모든 것의 평균으로 설정. -> 잔차를 구하고 잔차를 기준으로 decision tree 생성.
- 잔차에 learning rate를 곱하여서 예측값을 update, -> overfitting을 방지
- 전체 데이터에 대해 모든 잔차를 다 업데이트하고, 이를 기반으로 새로운 tree 생성
https://dailyheumsi.tistory.com/116

## XGboost
- GBM 기반의 알고리즘으로 분산 환경에서도 실행할 수 있도록 구현해놓은 라이브러리
- XGboost 장점
    - 뛰어난 예측 성능
    - GBM 대비 빠른 수행 시간
    - 과적합 규제
    - Tree pruning
    - 자체 내장된 교차 검증
    - 결손값 자체 처리

In [1]:
import pandas as pd
def get_human_dataset():
    feature_name_df = pd.read_csv("C:/Users/이혜림/Desktop/Bita5/9주차/UCI HAR Dataset/human_activity/features.txt",
                                 sep="\s+", header=None, names = ["column_index","column_name"])
    
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    feature_name = new_feature_name_df.iloc[:.1].values.tolist()
    # 피러명을 열로 부여하기 위해 리스트 객체로 변화
    
    X_train=pd.read_csv("C:/Users/이혜림/Desktop/Bita5/9주차/UCI HAR Dataset/human_activity/X_train.txt",
                        sep="\s+", names=feature_name)
    X_test = pd.read_csv("C:/Users/이혜림/Desktop/Bita5/9주차/UCI HAR Dataset/human_activity/X_test.txt",
                         sep="\s+", names=feature_name)
    y_train = pd.read_csv("C:/Users/이혜림/Desktop/Bita5/9주차/UCI HAR Dataset/human_activity/y_train.txt",
                          sep="\s+", header=None, names=["action"])
    y_test = pd.read_csv("C:/Users/이혜림/Desktop/Bita5/9주차/UCI HAR Dataset/human_activity/y_test.txt",
                         sep="\s+", header=None, names=["action"])
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = get_human_dataset()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings
warnings.filterwarnings("ignore")

X_train, X_test, y_train, y_test = get_human_dataset()

# GBM 수행 시간 측정을 위한 시작시간 설정
start_time = time.time()

# 예시 데이터셋 불러오기
gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train.values)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print(gb_accuracy)
print(time.time()-start_time)

In [ ]:
from sklearn.model_selection import GrdiSearchCV

params = {"n_estimators":[10,20],"learning_rate":[0.05,0.1]}

grid_cv = GridSearchCV(gb_clf, param_grid=params, cv=2, verbose=1)
grid_cv.fit(X_train, y_train)

print(grid_cv.best_params_)
print(grid_cv.best_scrore_)

In [ ]:
# GridSearchCV를 이용해 최적으로 학습된 estimators로 예측 수행
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print(gb_accuracy)

In [6]:
import xgboost as xgb
from xgboost import plot_importance
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

dataset = load_breast_cancer()
X_features = dataset.data
y_label =dataset.target

cancer_df = pd.DataFrame(data=X_features, columns=dataset.feature_names)
cancer_df["target"]=y_label

In [7]:
X_train, X_test, y_train, y_test=train_test_split(X_features,y_label,
                                                 test_size=0.2, random_state=156)

# Dmatrix : numpy 입력 파라미터를 받아서 만들어지는 XGBoost만의 전용 데이터셋
# pandas의 데이터셋이라면 numpy의 데이터셋으로 변경해주는 것이 필요함
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)

params = {"max_depth":3, "eta":0.1,"objective":"binary:logistic",
         "eval_metric":"logloss"}
num_rounds = 400

In [ ]:
wlist = [(dtrain,"train"),(dtest,"eval")]

# 하이퍼 파라미터와 early stopping 파라미터를 train()함수의 파라미터로 전달
xgb_model = xgb.train(params=params, dtrain=dtrain, num_boost_round=num_rounds,
                     early_stopping_rounds=100, evals=wlist)

# eval 파라미터에 학습 데이터 셋과 eval 데이터 셋을 명시해주면 eval 데이터셋에 대해
# 평가를 수행하면서 조기 중단 적용 가능


In [ ]:
pred_probs = xgb_model.predict(dtest)
print(np.round(pred_probs[:10],3))

preds = [1 if x>0.5 else 0 for x in pred_probs]
print(preds[:10])

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion=confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    # ROC-AUC 추가
    roc_auc = roc_auc_score(y_test, pred_proba)

In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10,12))
plot_importance(xgb_mdel, ax=ax)

In [ ]:
from xgboost import XGBClassifier

xgb_wrapper = XGBClassifier(n_estimaotrs=400, learning_rate = 0.1, max_depth=3)
xgb_wrapper.fit(X_train, y_train)
w_preds = xgb_wrapper.predict(X_test)
w_pred_proba = xgb_wrapper.predict_proba(X_test)[:,1]

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion=confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    # ROC-AUC 추가
    roc_auc = roc_auc_score(y_test, pred_proba)

In [ ]:
xgb_wrapper = XGBClassifier(n_estimaotrs=400, learning_rate = 0.1, max_depth=3)
xgb_wrapper.fit(X_train, y_train, early_stopping_rounds=100)
w_preds = xgb_wrapper.predict(X_test)
w_pred_proba = xgb_wrapper.predict_proba(X_test)[:,1]

# early stooping roudns -> 100번의 반복동안 더이상 성능 평가 지수가 향상되지 않는다면
# 반복을 멈춤

# 만약 100보다 더 적은 숫자로 하면 성능 저하가 많이 일어날 가능성이 o

In [ ]:
from xgboost import XGBClassifier

xgb_wrapper = XGBClassifier(n_estimaotrs=400, learning_rate = 0.1, max_depth=3)
xgb_wrapper.fit(X_train, y_train)
w_preds = xgb_wrapper.predict(X_test)
w_pred_proba = xgb_wrapper.predict_proba(X_test)[:,1]

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion=confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    # ROC-AUC 추가
    roc_auc = roc_auc_score(y_test, pred_proba)

In [8]:
from lightgbm import LGBMClassifier

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

dataset = load_breast_cancer()
ftr = dataset.data
target = dataset.target

X_train, X_test, y_train, y_test = train_test_split(ftr, target, test_size = 0.2,
                                                   random_state = 156)

# 트리 개수는 400개로 지정
lgbm_wrapper = LGMBClassifier(n_estimators=400)

# early stopping 기능
evals = [(X_test, y_test)]
lgbm_wrapper.fit(X_train,y_train, early_stopping_rounds = 100, eval_metric = "logloss",
                eval_set=evals, verbose=True)

preds = lgbm_wrapper.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion=confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    # ROC-AUC 추가
    roc_auc = roc_auc_score(y_test, pred_proba)